In [ ]:
pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.5/920.5 kB 20.2 MB/s eta 0:00:00


In [ ]:
# data1 = pd.read_csv('Bitcoin_modified')
# data1 = data1.drop('Unnamed: 0', axis=1)
# returns = abs(100 * data1['Close'].pct_change().fillna(0))
# ten_day_vol = []
# for i in range(0, 4419):
#     ten_day_vol.append(returns[i:i+10].std())
# data1['ten_day_vol'] = pd.Series(ten_day_vol, index = range(0,4419))
# data1 = data1.fillna(0)
# data1.isnull().values.any()
# # Split the data into train and test sets
# train_data, test_data = train_test_split(data1, test_size=0.2, shuffle=False)
# features = (train_data[['Garch Estimation', 'Previous 30 day Volatility', 'Previous 10 day Volatility', 'Log Return', 'Log Trading Range', 'Log Volume Change']])
# target = train_data['ten_day_vol']
# target2 = test_data['ten_day_vol']
# # # Define the number of input columns
# n_input = 6

# # # Define the number of hidden units in the LSTM layer
# n_units = 10

# # # Define the optimizer
# optimizer = Adam(lr=0.0)

# # # Define the loss function
# loss = 'mean_squared_error'

# # # Define the LSTM model
# model = Sequential()
# model.add(LSTM(units=n_units, input_shape=(n_input, 1)))
# model.add(Dense(units=1))

# # # Compile the model
# model.compile(optimizer=optimizer, loss=loss)

# #Reshape the training data
# X_train = np.array(train_data.iloc[:, 7:13]).reshape(-1, n_input, 1)

# y_train = np.array(target)
# #Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=50)

# # # Reshape the test data
# X_test = np.array(test_data.iloc[:, 7:13]).reshape(-1, n_input, 1)
# y_test = np.array(target2)

# # # Evaluate the model on the test data
# test_loss = model.evaluate(X_test, y_test)
# print(f'Test loss: {test_loss}')
# # data1

SyntaxError: ignored

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from arch.__future__ import reindexing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Activation, LSTM
from keras.optimizers import Adam
import keras

In [ ]:
csv_path = "https://raw.githubusercontent.com/curiousily/Deep-Learning-For-Hackers/master/data/3.stock-prediction/BTC-USD.csv"
df = pd.read_csv(csv_path, parse_dates=['Date'])
df = df.sort_values('Date')
scaler = MinMaxScaler()

close_price = df.Close.values.reshape(-1, 1)

scaled_close = scaler.fit_transform(close_price)
scaled_close = scaled_close[~np.isnan(scaled_close)]
scaled_close = scaled_close.reshape(-1, 1)
SEQ_LEN = 100

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test =\
 preprocess(scaled_close, SEQ_LEN, train_split = 0.95)
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1
model = keras.Sequential()
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=True),
  input_shape=(WINDOW_SIZE, X_train.shape[-1])
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM((WINDOW_SIZE * 2), return_sequences=True)
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM(WINDOW_SIZE, return_sequences=False)
))

model.add(Dense(units=1))

model.add(Activation('linear'))
BATCH_SIZE = 64

model.compile(
    loss='mean_absolute_error',
    optimizer='adam'
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
)

y_hat = model.predict(X_test)
y_test_inverse = scaler.inverse_transform(y_test)
y_hat_inverse = scaler.inverse_transform(y_hat)

Epoch 1/50
42/42 [==============================] - 92s 2s/step - loss: 0.0254 - val_loss: 0.6280
Epoch 2/50
42/42 [==============================] - 77s 2s/step - loss: 0.0458 - val_loss: 0.0681
Epoch 3/50
42/42 [==============================] - 75s 2s/step - loss: 0.0338 - val_loss: 0.5509
Epoch 4/50
42/42 [==============================] - 75s 2s/step - loss: 0.0313 - val_loss: 0.0700
Epoch 5/50
42/42 [==============================] - 75s 2s/step - loss: 0.0421 - val_loss: 0.2082
Epoch 6/50
42/42 [==============================] - 73s 2s/step - loss: 0.0145 - val_loss: 0.1163
Epoch 7/50
42/42 [==============================] - 75s 2s/step - loss: 0.0350 - val_loss: 0.2968
Epoch 8/50
42/42 [==============================] - 75s 2s/step - loss: 0.0393 - val_loss: 0.2105
Epoch 9/50
42/42 [==============================] - 74s 2s/step - loss: 0.0198 - val_loss: 0.0294
Epoch 10/50
42/42 [==============================] - 78s 2s/step - loss: 0.0318 - val_loss: 0.1233
Epoch 11/50
42/42 [

In [ ]:
csv_path = "https://raw.githubusercontent.com/curiousily/Deep-Learning-For-Hackers/master/data/3.stock-prediction/BTC-USD.csv"
df = pd.read_csv('bitcoin.csv', parse_dates=['Date'])
df = df.sort_values('Date')
scaler = MinMaxScaler()

close_price = df.Close.values.reshape(-1, 1)

scaled_close = scaler.fit_transform(close_price)
scaled_close = scaled_close[~np.isnan(scaled_close)]
scaled_close = scaled_close.reshape(-1, 1)
SEQ_LEN = 100

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test =\
 preprocess(scaled_close, SEQ_LEN, train_split = 0.95)
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1
model = keras.Sequential()
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=True),
  input_shape=(WINDOW_SIZE, X_train.shape[-1])
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM((WINDOW_SIZE * 2), return_sequences=True)
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM(WINDOW_SIZE, return_sequences=False)
))

model.add(Dense(units=1))

model.add(Activation('linear'))
BATCH_SIZE = 64

model.compile(
    loss='mean_absolute_error',
    optimizer='adam'
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
)

y_hat = model.predict(X_test)
y_test_inverse = scaler.inverse_transform(y_test)
y_hat_inverse = scaler.inverse_transform(y_hat)

Epoch 1/50
58/58 [==============================] - 75s 1s/step - loss: 0.0163 - val_loss: 0.2505
Epoch 2/50
58/58 [==============================] - 62s 1s/step - loss: 0.0170 - val_loss: 0.0924
Epoch 3/50
58/58 [==============================] - 62s 1s/step - loss: 0.0147 - val_loss: 0.3943
Epoch 4/50
58/58 [==============================] - 63s 1s/step - loss: 0.0144 - val_loss: 0.0540
Epoch 5/50
58/58 [==============================] - 62s 1s/step - loss: 0.0186 - val_loss: 0.3807
Epoch 6/50
58/58 [==============================] - 62s 1s/step - loss: 0.0165 - val_loss: 0.1463
Epoch 7/50
58/58 [==============================] - 62s 1s/step - loss: 0.0199 - val_loss: 0.4955
Epoch 8/50
58/58 [==============================] - 63s 1s/step - loss: 0.0094 - val_loss: 0.0496
Epoch 9/50
58/58 [==============================] - 62s 1s/step - loss: 0.0124 - val_loss: 0.0858
Epoch 10/50
58/58 [==============================] - 62s 1s/step - loss: 0.0102 - val_loss: 0.2467
Epoch 11/50
58/58 [

In [ ]:
arr = []
for i in range(0, len(y_hat_inverse)-10):
        arr.append((y_hat_inverse[i:i+10].std()))

In [ ]:
arr1 = []
for i in range(0, len(y_test_inverse)-10):
        arr1.append((y_test_inverse[i:i+10].std()))

In [33]:
arr=np.array(arr)
arr1=np.array(arr1)
abs((1 - (arr1/arr))).mean()

0.31780056590387734

In [35]:
print(type(y_hat_inverse))

<class 'numpy.ndarray'>


In [37]:
file = open("filebitcoin.txt", "w+")
 
# Saving the array in a text file
content = str(y_hat_inverse)
file.write(content)
file.close()